In [1]:
from qite import *
from helper import *
%matplotlib inline
import matplotlib.pyplot as plt

from qiskit.quantum_info import Statevector

In [44]:
qc = QuantumCircuit(3)
qbits = [0,1,2]
p = 16*1 + 4*2 + 1*3

# def ib4(x, nbit):
#     string = [0] * nbit
#     for i in range(nbit):
#         print(x,x//4,x%4)
#         string[-(i+1)] = x % 4
#         x = x//4
#     return string

# print(ib4(p,3))


# initialize state
zero    = np.array([1,0],dtype=complex)
one     = np.array([0,1],dtype=complex)
plus    = np.array([1,1],dtype=complex)   * np.sqrt(0.5)
minus   = np.array([1,-1],dtype=complex)  * np.sqrt(0.5)
plus_i  = np.array([1,1j],dtype=complex)  * np.sqrt(0.5)
minus_i = np.array([1,-1j],dtype=complex) * np.sqrt(0.5)

theta = np.pi/2
nbit = len(qbits)
pstring = int_to_base4(p,nbit)
print(pstring)
cnots = []

for i in range(nbit):
    if pstring[i] == 1:
        qc.h(qbits[i])
#         qc.ry(-np.pi/2,qbits[i])
    elif pstring[i] == 2:
        qc.rx(np.pi/2, qbits[i])
        qc.s(qbits[i])
    elif pstring[i] == 3:
        None
    else: # pstring[i] == 0, don't add this to the list of cnots
        continue
    cnots.append(qbits[i])

print(cnots)

for i in range(len(cnots)-1):
    qc.cx(cnots[i],cnots[i+1])

qc.rz(theta, cnots[-1])

for i in range(len(cnots)-2,-1,-1):
    qc.cx(cnots[i], cnots[i+1])

for i in range(nbit):
    if pstring[i] == 1:
        qc.h(qbits[i])
#         qc.ry(np.pi/2, qbits[i])
    elif pstring[i] == 2:
        qc.sdg(qbits[i])
        qc.rx(np.pi/2, qbits[i])
    else:
        None
    
qc.draw()

[1, 2, 3]
[0, 1, 2]


┌───┐                                        ┌───┐            
q_0: ───┤ H ├──────────■─────────────────────────■───┤ H ├────────────
     ┌──┴───┴──┐┌───┐┌─┴─┐                     ┌─┴─┐┌┴───┴┐┌─────────┐
q_1: ┤ Rx(π/2) ├┤ S ├┤ X ├──■───────────────■──┤ X ├┤ Sdg ├┤ Rx(π/2) ├
     └─────────┘└───┘└───┘┌─┴─┐┌─────────┐┌─┴─┐└───┘└─────┘└─────────┘
q_2: ─────────────────────┤ X ├┤ Rz(π/2) ├┤ X ├───────────────────────
                          └───┘└─────────┘└───┘

In [45]:
init = QuantumCircuit(3)

init.initialize(plus, 0)
init.initialize(plus_i, 1)
init.initialize(zero, 2)

new_sv = Statevector.from_label('000')
init_sv = new_sv.evolve(init)
evolved_sv = init_sv.evolve(qc)
print(init_sv.data)
final_data = evolved_sv.data #/ np.exp(-1j*theta/2)
# print(evolved_sv.data)
# print(final_data)
fd = [d if np.abs(d) > 1e-5 else 0 for d in final_data]
print(fd)

[0.5+0.j  0.5+0.j  0. +0.5j 0. +0.5j 0. +0.j  0. +0.j  0. +0.j  0. +0.j ]
[(0.3535533905932738-0.3535533905932738j), (0.3535533905932738-0.3535533905932738j), (-0.3535533905932738-0.3535533905932738j), (-0.3535533905932738-0.3535533905932738j), 0, 0, 0, 0]


In [46]:
print(fd / np.exp(-1j*theta/2))

[0.5+0.j  0.5+0.j  0. -0.5j 0. -0.5j 0. +0.j  0. +0.j  0. +0.j  0. +0.j ]
